# Ishigami fonction
Nous allons travailler avec la fonction d'Ishigami :
$f(X_1,X_2,X_3)=sin(X_1)+7*sin^2(X_2)+0.1{X{_3}^4}sin(X_1)$
avec X_i uniforme sur $[-\pi,\pi]$, pour i=1,2,3

In [97]:
def LHS(N,dimension=3,quant_samples=10):
    """ Cette fonction va tirer les LHS """
    permutation_array= permutation(N,dimension)
    
    cellules=create_cellules(N,dimension,permutation_array)
    
    cellules_final={}
    cellules_final["cellules"]=cellules.copy()
    cellules_final["distancia_minima"]=distance_minimale(cellules)
    
    for count in np.arange(quant_samples):
        cellules=create_cellules(N,dimension,permutation_array)
        distancia=distance_minimale(cellules)
        if(distancia>cellules_final["distancia_minima"]):
            cellules_final["cellules"]=cellules.copy()
            cellules_final["distancia_minima"]=distancia

    return cellules_final
        
            
     
def create_cellules(N,dimension,permutation):
        cellule={}
        for i in np.arange(N):
            cellule[i]=[] #on choisit la i-ème 
            for d in np.arange(dimension):
            #1/N(random.random()+permutation[d][i]) choisit l'element dans la cellule qui après est prise redimensionné 
            #sur [-pi,pi]
                   cellule[i].append(2*pi*(1/N)*(np.random.random()+permutation[d][i])-pi) 
        return cellule
    
def permutation(N,dimension):
        permutation={}
        lista=np.arange(N)
   
        for i in range(dimension):
            random.shuffle(lista)
            permutation[i]=lista
            lista=np.arange(N)
            
        return permutation
    
def distance(x,y):

        z=[(x[i]-y[i])**2 for i in range(len(x))]
        return np.sqrt(reduce(lambda x,y: x+y, z))

def distance_minimale(cellules):
        i=len(cellules.keys())
        distance_minimale=2*pi*np.sqrt(i)
        for k in range(i):
            j=k+1
            while(j<i):

                distance_temp=distance(cellules[k],cellules[j])
                if(distance_temp<distance_minimale):
                    distance_minimale=distance_temp
                j+=1
        return distance_minimale
            
    

In [98]:
print(LHS(10,3))

{'cellules': {0: [3.0003436675533663, -2.989716157204549, 1.4994126921736788], 1: [2.3021808488779936, 3.0101176923242905, -1.7538351458419124], 2: [-0.08617669410457918, 0.8720198130510219, -0.9749362146183258], 3: [-1.4721238616098802, 0.022406859419778602, 3.030876235277117], 4: [0.6207199027919548, -1.4073623447771317, -2.3461836669091647], 5: [1.1450827320060393, -0.12608981199395508, 0.34302326911998593], 6: [-1.0817173811643657, -1.1503177744409767, -3.079633761591402], 7: [-2.1801355951887214, 1.287765400443769, -0.5561332407274575], 8: [-2.787883433332201, 2.3164664788514804, 0.9883152851327264], 9: [1.5570754731984424, -2.0396743913672397, 2.2754009766230574]}, 'distancia_minima': 1.8714469424307103}


In [99]:
dic={}
for i in np.arange(60,120,10):
    dic[i]=LHS(i,3,20)

# Pour la construction du metamodèle 
En utilisant la fonction Matern5/2 fournis sur le poly page 102.


In [94]:
def matern(x,y,sigma_mod,l):
    return (sigma**2)*(1+np.sqrt(5)*abs(distance(x,y))/l+5*(distance(x,y))**2/(3*l**2))*np.exp(-1*np.sqrt(5)*distance(x,y)/l)



def g_func(x):
    p1 = 4*np.abs(2*x[0]-1) - 1
    p2 = 2*np.abs(2*x[1]-1)
    p3 = (4*np.abs(2*x[2]-1) + 1)/3
    return p1*p2*p3


def f0(x):
    p2 = np.abs(2*x[1]-1)
    return -2*p2/3


def f1(x):
    p1 = np.abs(2*x[0]-1)
    p2 = np.abs(2*x[1]-1)
    p3 = np.abs(2*x[2]-1)
    return p1*p2*p3


def f2(x):
    p1 = np.abs(2*x[0]-1)
    p2 = np.abs(2*x[1]-1)
    return p1*p2


def f3(x):
    p2 = np.abs(2*x[1]-1)
    p3 = np.abs(2*x[2]-1)
    return p2*p3


def identity(x, y):
    if x == y:
        return 1
    else:
        return 0

def ishigami(x):
    return np.sin(x[0]){1+b*x[2]**4}+7*np.sin(x[1])**2
    
    

def PG_regression(training_data, sigma_mes, sigma_mod, lc):
    N = len(training_data)

    yreal = np.zeros(N)
    y0 = np.zeros(N)
    H = np.zeros((N, 3))
   

    for i in range(N):
        x = training_data[i]
        yreal[i] = g_func(x)
        y0[i] = f0(x)
        H[i][0] = f1(x)
        H[i][1] = f2(x)
        H[i][2] = f3(x)

    eps = sigma_mes*np.random.randn(N)
    yobs = yreal + eps

    R = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            R[i][j] = identity(i, j)*sigma_mes**2 + \
                matern(training_data[i], training_data[j], sigma_mod, lc)
            R[j][i] = R[i][j]

    R_inv = np.linalg.inv(R)
    H_trans = H.T

    Q_inv = (H_trans.dot(R_inv)).dot(H)
    Q_post = np.linalg.inv(Q_inv)

    H_R_y = (H_trans.dot(R_inv)).dot(yobs-y0)
    beta_post = Q_post.dot(H_R_y)

    return yreal, yobs, y0, H, R, beta_post, Q_post

2.b)Validation croisee


2.c) Base de test

In [61]:
base_de_test_ishigami=[]
qtde=100
for i in range(qtde):
    base_de_test.append(2*pi*np.random.random(3)-pi)


[array([ 1.1359893 , -0.93343827, -0.71782012]),
 array([-0.46462424, -2.30735521,  1.05801863]),
 array([ 0.27508144, -1.43808445, -2.49547543]),
 array([ 2.26483647, -3.0427805 ,  0.56745317]),
 array([-1.77989042, -0.6546467 ,  0.74003662]),
 array([ 1.89652712, -0.50456702,  0.02434623]),
 array([-1.50759029, -2.58039386,  1.09864386]),
 array([-2.41258361, -2.28153665, -1.89395505]),
 array([ 1.67983445, -2.65681728,  0.10607954]),
 array([ 2.07475334,  1.1245156 , -1.68082719]),
 array([-1.04993927, -1.72271141, -0.54750904]),
 array([2.71332229, 0.90688608, 3.02606299]),
 array([ 1.61491943, -1.56202063, -0.63693674]),
 array([ 1.31860392, -0.91233562, -0.51554377]),
 array([-1.84951332,  0.02072798,  0.40722106]),
 array([-0.79144938, -1.50015688, -0.3966058 ]),
 array([ 1.58764829,  2.21808883, -1.42275343]),
 array([-0.39230248,  0.29997705,  2.85739355]),
 array([ 0.76940593, -0.26985196, -1.21604224]),
 array([-1.71090494,  1.57127512, -0.88023521]),
 array([-1.20582572,  1

# Calcul analytique des indices de Sobol

Indices de Sobol du 1er ordre théorique:
$f(X_1,X_2,X_3)=sin(X_1)+a*sin^2(X_2)+b{X{_3}^4}sin(X_1)$
<br>$\mathbb{V}ar(Y)=\frac{a^2}{8}+b*\frac{\pi^4}{5}+\frac{b^2\pi^8}{18}+\frac{1}{2}$
<br>$V_1=1/2 * (1+\frac{\pi^4}{5})^2 $
<br> $V_2=a^2/8$
<br>$V_3=0 = V_{1,2}=V{2,3}$
<br>$V_{1,3}=\frac{8b^2\pi^8}{225}$
<br>$S_i=V_i/Var(Y)$
<br>$S_{total:1}=S_1+S_{1,3}$
<br>$S_{total:2}=S_2$
<br>$S_{total:3}=S_{1,3}$

In [88]:
def valeur_analytique(a,b):
    var=(a**2)/8+b*pi**4/5+b**2*((pi**8)/18)+1/2
    print(var)
    v_1=1/2*(1+b*pi**4/5)**2
    v_2=a**2/8
    v_3=0
    return var,v_1,v_2,v_3
def v13(b):
    return 8*b**2*pi**8/225

In [89]:
a,b=(7,.1)
var,v_1,v_2,v_3=valeur_analytique(a,b)
S=[]
print(var,v_1,v_2,v_3)
S.append(v_1/var)
S.append(v_2/var)
S.append(v_3/var)
print("S de première ordre est",S)
S_total=S[:]

S_total[0]=(S[0]+v_13/var)
S_total[2]=(S_total[2]+v_13/var)

print("S total de Sobol est",S_total)

13.844545738067614
13.844545738067614 4.345868618314417 6.125 0
S de première ordre est [0.313904746355441, 0.442412493402251, 0.0]
S total de Sobol est [0.5575875065977491, 0.442412493402251, 0.24368276024230806]


# 5) Calcul des indices du 1er ordre et totaux utilisant PG

In [ ]:
#Calcul des

In [ ]:
#comparaison

# La decomposition de Hoeffding-Sobol est:
<br> $f_{meta}=f_0+ f_1(X_1)+f_2(X_2)+f_3(X_3)+f_{1,2}(X_1,X_2)+f_{1,3}(X_1,X_3)+f_{2,3}(X_2,X_3)+f_{1,2}(X_1,X_2,X_3)$
<br> $\mathbb{E}(f) = \frac{7}{2} = f_0$
<br> $f_{i,j,k}=\mathbb{E}(f(X_a,X_b,X_C)|X_i,X_j,X_k)$
<br>En faisant les calculs:<br>
<br>$f_0 = \frac{7}{2}$
<br>$f_1 = \frac{\pi^4sin(X_1)}{50} + sen(X_1) - \frac{7}{2} $
<br>$f_2 = 7sin^2(X_2)- \frac{7}{2}$
<br>$f_3 = -\frac{7}{2}$
<br>$f_{1,2}= +7/2$
<br>$f_{1,3}=sin(X_1)*\frac{5X_3^4-\pi^4 }{50}+7/2$
<br>$f_{2,3}=7/2$
<br>$f_{1,2,3}=-7/2$

<br>
Calcul des indices de Sobol totaux:
<br>
<br>
